In [47]:
import requests
import lxml.html as lh
import pandas as pd

In [48]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [49]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

Parsing the raw html data 

In [50]:
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


Creating Pandas DataFrame
Each header is appended to a tuple along with an empty list.

In [51]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [52]:
[len(C) for (title,C) in col]

[181, 181, 181]

In [53]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [54]:

# Access the top 5 rows of the data frame 
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [55]:

df.columns = ['Postal Code', 'Borough','Neighbourhood']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

df.head()

,Neighbourhood,Postal Code,Borough
0,Not assigned\n,M1A\n,Not assigned\n
1,Not assigned\n,M2A\n,Not assigned\n
2,Parkwoods\n,M3A\n,North York\n
3,Victoria Village\n,M4A\n,North York\n
4,"Regent Park, Harbourfront\n",M5A\n,Downtown Toronto\n


Removing "/n"

In [56]:
df = df.replace('\n',' ', regex=True)
df.head()

,Neighbourhood,Postal Code,Borough
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,"Regent Park, Harbourfront",M5A,Downtown Toronto


In [57]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,Neighbourhood,Postal Code,Borough
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,"Regent Park, Harbourfront",M5A,Downtown Toronto
5,"Lawrence Manor, Lawrence Heights",M6A,North York
6,"Queen's Park, Ontario Provincial Government",M7A,Downtown Toronto
7,Not assigned,M8A,Not assigned
8,"Islington Avenue, Humber Valley Village",M9A,Etobicoke
9,"Malvern, Rouge",M1B,Scarborough


In [58]:
df = df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postal Code','Borough','Neighbourhood']
df.head(10)

,Postal Code,Borough,Neighbourhood
0,,Canadian postal codes,
1,M1A,Not assigned,Not assigned
2,M1B,Scarborough,"Malvern, Rouge"
3,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
4,M1E,Scarborough,"Guildwood, Morningside, West Hill"
5,M1G,Scarborough,Woburn
6,M1H,Scarborough,Cedarbrae
7,M1J,Scarborough,Scarborough Village
8,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
9,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"


Dropping header

In [59]:
df.drop(df.index[0],inplace=True)

In [70]:
df = df[df['Borough']!= 'Not assigned ']


In [72]:
set(df.Borough)

{'Central Toronto ',
 'Downtown Toronto ',
 'East Toronto ',
 'East York ',
 'Etobicoke ',
 'Mississauga ',
 'North York ',
 'Scarborough ',
 'West Toronto ',
 'York '}

In [74]:
df = df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [75]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

In [78]:
df.loc[df['Neighbourhood'] == 'Not assigned ', 'Neighbourhood'] = df['Borough']

In [79]:
# Check if the Neighbourhood for Queen's Park changed 
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighbourhood


In [82]:
'Not assigned ' in set(df.Neighbourhood)

False

In [83]:
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
